In [3]:
from datetime import datetime
from transformation_handler import readData
import pandas as pd

In [4]:
dfs = readData(limit=1000000)

In [5]:
dfs.keys()

dict_keys(['shelter_sonoma', 'shelter_austin_intakes', 'shelter_austin_outcomes', 'shelter_norfolk', 'shelter_bloomington', 'shelter_dallas_2017_2018', 'shelter_dallas_2018_2019', 'shelter_dallas_2019_2020', 'shelter_dallas_2020_2021', 'shelter_dallas_2021_2022', 'shelter_dallas_2022_2023', 'per_capita_sonoma_income', 'per_capita_austin_income', 'per_capita_norfolk_income', 'per_capita_bloomington_income', 'per_capita_dallas_income'])

In [6]:
for key, df in dfs.items():
    print(f"DataFrame: {key}")
    print(df.info())

DataFrame: shelter_sonoma
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26315 entries, 0 to 26314
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             26315 non-null  object
 1   type           26315 non-null  object
 2   breed          26315 non-null  object
 3   color          26315 non-null  object
 4   sex            26315 non-null  object
 5   date_of_birth  19827 non-null  object
 6   intake_date    26315 non-null  object
 7   outcome_date   26069 non-null  object
 8   intake_type    26315 non-null  object
 9   outcome_type   26064 non-null  object
dtypes: object(10)
memory usage: 2.0+ MB
None
DataFrame: shelter_austin_intakes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156135 entries, 0 to 156134
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   animal_id    156135 non-null  object
 1   animal_type  156135 non-null

In [7]:
sonoma = dfs['shelter_sonoma']
sonoma.columns

Index(['id', 'type', 'breed', 'color', 'sex', 'date_of_birth', 'intake_date',
       'outcome_date', 'intake_type', 'outcome_type'],
      dtype='object')

In [8]:
sonoma.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26315 entries, 0 to 26314
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             26315 non-null  object
 1   type           26315 non-null  object
 2   breed          26315 non-null  object
 3   color          26315 non-null  object
 4   sex            26315 non-null  object
 5   date_of_birth  19827 non-null  object
 6   intake_date    26315 non-null  object
 7   outcome_date   26069 non-null  object
 8   intake_type    26315 non-null  object
 9   outcome_type   26064 non-null  object
dtypes: object(10)
memory usage: 2.0+ MB


In [27]:
sonomaa = sonoma.copy()
sonomaa = sonomaa.drop_duplicates()
date_columns = ['intake_date','date_of_birth','outcome_date']
sonomaa[date_columns] = sonomaa[date_columns].apply(pd.to_datetime)
# sonomaa[['date_of_birth','outcome_date']] = sonomaa[['date_of_birth','outcome_date']].apply(lambda x: x.fillna(pd.NaT))
sonomaa.dropna(subset=['intake_date','intake_type'], inplace=True)
sonomaa['outcome_type'] = sonomaa['outcome_type'].fillna('Pending')
sonomaa['id'] = sonomaa['id'].apply(lambda x: f'SO-{x}')
sonomaa['days_in_shelter'] = (sonomaa['outcome_date'] - sonomaa['intake_date']).dt.days
# sonomaa['days_in_shelter'].fillna((datetime.now() - sonomaa['intake_date']).dt.days,inplace=True)
# sonomaa['days_in_shelter'] = sonomaa['days_in_shelter'].astype(int)
data_columns = ['type', 'breed', 'color', 'sex', 'intake_type', 'outcome_type']
sonomaa[data_columns] = sonomaa[data_columns].apply(lambda x: x.str.title())
sonomaa['region'] = 'Sonoma'
sonomaa.loc[sonomaa['outcome_type'] == 'Rtos', 'outcome_type'] = 'Return To Owner'
sonomaa['color'] = sonomaa['color'].replace({'Bl ': 'Black ', 'Brn ' : 'Brown '},regex=True)
sonomaa.head()

,id,type,breed,color,sex,date_of_birth,intake_date,outcome_date,intake_type,outcome_type,days_in_shelter,region
0,SO-A328255,Dog,Maltese/Poodle Toy,White,Spayed,2014-10-06,2023-07-05,2023-08-08,Stray,Adoption,34.0,Sonoma
1,SO-A420799,Cat,Domestic Sh,Black,Unknown,NaT,2023-09-30,2023-09-30,Stray,Transfer,0.0,Sonoma
2,SO-A420773,Cat,Domestic Lh,Black/White,Neutered,2013-09-29,2023-09-29,2023-09-30,Stray,Return To Owner,1.0,Sonoma
3,SO-A420810,Cat,Domestic Sh,Brown Tabby,Male,2023-08-15,2023-09-30,2023-09-30,Stray,Transfer,0.0,Sonoma
4,SO-A417889,Cat,Domestic Sh,Org Tabby/White,Spayed,2023-05-07,2023-05-30,2023-08-08,Stray,Adoption,70.0,Sonoma


In [25]:
sonomaa.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26313 entries, 0 to 26314
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id               26313 non-null  object        
 1   type             26313 non-null  object        
 2   breed            26313 non-null  object        
 3   color            26313 non-null  object        
 4   sex              26313 non-null  object        
 5   date_of_birth    19826 non-null  datetime64[ns]
 6   intake_date      26313 non-null  datetime64[ns]
 7   outcome_date     26067 non-null  datetime64[ns]
 8   intake_type      26313 non-null  object        
 9   outcome_type     26313 non-null  object        
 10  days_in_shelter  26067 non-null  float64       
 11  region           26313 non-null  object        
dtypes: datetime64[ns](3), float64(1), object(8)
memory usage: 2.6+ MB


In [11]:
sonomaa.intake_type.value_counts()

Stray              19207
Owner Surrender     3234
Confiscate          2100
Quarantine           864
Adoption Return      435
Transfer             432
Born Here             38
Os Appt                3
Name: intake_type, dtype: int64

In [12]:
sonomaa.outcome_type.value_counts()

Return To Owner    8810
Adoption           7686
Transfer           5536
Euthanize          3599
Pending             251
Disposal            237
Died                177
Escaped/Stolen       17
Name: outcome_type, dtype: int64

In [13]:
sonomaa.sex.value_counts()

Neutered    8606
Spayed      7117
Male        4506
Female      3733
Unknown     2351
Name: sex, dtype: int64

In [14]:
sonomaa.color.nunique()

344

In [15]:
sonomaa.breed.nunique()

1052

In [16]:
sonomaa.type.value_counts()

Dog      14701
Cat       9408
Other     2204
Name: type, dtype: int64

In [17]:
sonomaa.columns

Index(['id', 'type', 'breed', 'color', 'sex', 'date_of_birth', 'intake_date',
       'outcome_date', 'intake_type', 'outcome_type', 'days_in_shelter',
       'region'],
      dtype='object')

In [18]:
sonomaa.intake_type.value_counts()

Stray              19207
Owner Surrender     3234
Confiscate          2100
Quarantine           864
Adoption Return      435
Transfer             432
Born Here             38
Os Appt                3
Name: intake_type, dtype: int64

In [28]:
sonomaa.color.value_counts(ascending=True)

Black Brindle/Black       1
Black/Brown Merle         1
Buff/Tan                  1
Apricot/White             1
Seal Pt/Brown             1
                       ... 
Black/Tan              1147
White                  1405
Brown Tabby            1413
Black                  2513
Black/White            2956
Name: color, Length: 344, dtype: int64